In [5]:
import geopandas as gpd
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
from tqdm import tqdm
import IPython.display as display
import copy
import seaborn as sns

In [15]:
def merge_csv_files(directory):
    # Get a list of all the csv files
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

    # Initialize an empty list to hold dataframes
    dfs = []

    # Loop through csv files, read each into a dataframe, and append to the list
    for file in csv_files:
        # Extract date from filename, assuming the date is in format 'traffic_flow_YYYY_MM_DD'
        date_str = file.split('.')[0].split('_')[-3:]  # This gives ['YYYY', 'MM', 'DD']
        date = datetime.strptime('_'.join(date_str), '%Y_%m_%d').date()

        df = pd.read_csv(os.path.join(directory, file))

        # Add date as a new column
        df['date'] = date.strftime('%m/%d/%y')

        dfs.append(df)

    # Concatenate all dataframes in the list into one dataframe
    merged_df = pd.concat(dfs, ignore_index=True).drop_duplicates()

    # Return the merged dataframe
    return merged_df


traffic_flows = merge_csv_files(
    '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/[XH]Traffic flow')
road_network = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/[XH]road_network/road_network.shp'

# clean the traffic flow data
traffic_flows = traffic_flows.drop_duplicates(['toid', 'date'])
traffic_flows = traffic_flows.groupby(['toid', 'date']).agg(
    {'bus': 'sum', 'car': 'sum', 'cycle': 'sum', 'walks': 'sum', 'stationary': 'sum'}).reset_index()

lsoa = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/London administrative boundaries/london_LSOA/london_LSOA.shp'

inoutter = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/London administrative boundaries/lp-consultation-oct-2009-inner-outer-london-shp/lp-consultation-oct-2009-inner-outer-london.shp'
# tube_line = 'https://raw.githubusercontent.com/oobrien/vis/master/tubecreature/data/tfl_lines.json'
# tube_station = 'https://raw.githubusercontent.com/oobrien/vis/master/tubecreature/data/tfl_stations.json'

inoutter = gpd.read_file(inoutter)
inoutter.to_crs(epsg=27700, inplace=True)

# tube_station = gpd.read_file(tube_station)
# tube_station.to_crs(epsg=27700, inplace=True)
# tube_station = gpd.sjoin(tube_station, inoutter, op='within')

# tube_line = gpd.read_file(tube_line)
# tube_line.to_crs(epsg=27700, inplace=True)
# tube_line = gpd.sjoin(tube_line, inoutter, op='within')

lsoa = gpd.read_file(lsoa, crs={'init': 'epsg:27700'})
road_network = gpd.read_file(road_network, crs={'init': 'epsg:27700'})
road_network.rename(columns={'NAME': 'boroughs'}, inplace=True)
road_network.loc[:, ['cycle_lane', 'bus_lane']] = road_network[['cycle_lane', 'bus_lane']].fillna('n')


# clean the traffic flow data
traffic_flows = traffic_flows.drop_duplicates(['toid', 'date'])
traffic_flows = traffic_flows.groupby(['toid', 'date']).agg(
    {'bus': 'sum', 'car': 'sum', 'cycle': 'sum', 'walks': 'sum', 'stationary': 'sum'}).reset_index()
traffic_flows['total'] = traffic_flows['bus'] + traffic_flows['car'] + traffic_flows['cycle'] + traffic_flows[
    'walks'] + traffic_flows['stationary']

flows = pd.merge(
    road_network[
        ['toid', 'roadclassi', 'geometry', 'cycle_lane', 'bus_lane', 'boroughs']],
    traffic_flows, left_on='toid', right_on='toid', how='left')
flows.set_geometry('geometry', inplace=True)

flows['classification'] = flows['roadclassi'].replace(
    {'Unknown': 'Local Road', 'Not Classified': 'Local Road', 'Unclassified': 'Local Road',
     'Classified Unnumbered': 'Local Road', 'A Road': 'Strategic Road', 'B Road': 'Strategic Road'})

flows.drop(columns=['roadclassi'], inplace=True)

stage_date = ['03/01/22', '02/22/22', '03/08/22']
flows = flows.loc[flows['date'].isin(stage_date)]

# label the regional level
flows = gpd.sjoin(flows, inoutter, how='inner', predicate='within')
flows = flows.drop(columns=['index_right', 'Source', 'Area_Ha', 'Shape_Leng', 'Shape_Area'])
flows.reset_index(drop=True, inplace=True)


merged = flows
flows = merged

# convert the dataframe
flows = pd.melt(flows,
                id_vars=['toid', 'classification', 'geometry', 'date', 'Boundary', 'cycle_lane', 'bus_lane',
                         'boroughs'],
                var_name='mode', value_name='flow')

flows = pd.pivot_table(flows,
                       index=['toid', 'classification', 'geometry', 'Boundary', 'mode', 'cycle_lane', 'bus_lane',
                              'boroughs'],
                       columns='date',
                       values='flow',
                       aggfunc='first').reset_index()


flows = flows.groupby(
    ['toid', 'mode', 'classification', 'geometry', 'Boundary', 'cycle_lane', 'bus_lane', 'boroughs'],
    as_index=False).agg(
    {'03/01/22': 'first', '02/22/22': 'first', '03/08/22': 'first'})
# Calculate the impact and recovery flows for one strike
flows['impact_flow'] = flows['03/01/22'] - flows['02/22/22']
flows['recovery_flow'] = flows['03/08/22'] - flows['03/01/22']

# Calculate impact rate while avoiding division by zero
flows['impact_rate'] = flows.apply(
    lambda row: round(row['impact_flow'] / row['02/22/22'], 4) if row['02/22/22'] != 0 else 0, axis=1)
# Calculate recovery rate while avoiding division by zero
flows['recovery_rate'] = flows.apply(
    lambda row: round(row['recovery_flow'] / row['03/01/22'], 4) if row['03/01/22'] != 0 else 0, axis=1)

All = flows.copy()
All

date,toid,mode,classification,geometry,Boundary,cycle_lane,bus_lane,boroughs,03/01/22,02/22/22,03/08/22,impact_flow,recovery_flow,impact_rate,recovery_rate
0,osgb4000000027865921,bus,Motorway,"LINESTRING (531539.442 200769.874, 531592.988 ...",Outer London,n,n,Enfield,16,11,12,5,-4,0.4545,-0.2500
1,osgb4000000027865921,car,Motorway,"LINESTRING (531539.442 200769.874, 531592.988 ...",Outer London,n,n,Enfield,1041,1100,1081,-59,40,-0.0536,0.0384
2,osgb4000000027865921,cycle,Motorway,"LINESTRING (531539.442 200769.874, 531592.988 ...",Outer London,n,n,Enfield,14,4,7,10,-7,2.5000,-0.5000
3,osgb4000000027865921,stationary,Motorway,"LINESTRING (531539.442 200769.874, 531592.988 ...",Outer London,n,n,Enfield,2,0,1,2,-1,0.0000,-0.5000
4,osgb4000000027865921,total,Motorway,"LINESTRING (531539.442 200769.874, 531592.988 ...",Outer London,n,n,Enfield,1095,1151,1122,-56,27,-0.0487,0.0247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1742635,osgb5000005242182149,car,Local Road,"LINESTRING (543548.945 179236.408, 543554.000 ...",Inner London,n,n,Greenwich,33,54,51,-21,18,-0.3889,0.5455
1742636,osgb5000005242182149,cycle,Local Road,"LINESTRING (543548.945 179236.408, 543554.000 ...",Inner London,n,n,Greenwich,2,1,1,1,-1,1.0000,-0.5000
1742637,osgb5000005242182149,stationary,Local Road,"LINESTRING (543548.945 179236.408, 543554.000 ...",Inner London,n,n,Greenwich,3,4,5,-1,2,-0.2500,0.6667
1742638,osgb5000005242182149,total,Local Road,"LINESTRING (543548.945 179236.408, 543554.000 ...",Inner London,n,n,Greenwich,50,74,70,-24,20,-0.3243,0.4000
